In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

# Set the working directory to the main directory
os.chdir(os.path.dirname(os.path.abspath('')))

# Verify the current working directory
print(os.getcwd())

/home/stef/projects/ai-amg


In [5]:
from music_gen.generator import MetaGenerator
from music_gen.controllers import AbletonOSCController, AbletonMetaController
# from music_gen import helpers

In [150]:
controller = AbletonOSCController()
controller.song.set_tempo(100)
# controller.device.set_parameter(2, 1, 14, 0.8) # controlling frequency of filter

# controller.track.set_volume(4, 0.5)
# controller.track.set_send(0, 2, 0)

In [4]:
# THE FOLLOWING IS TO FIND OUT THE INDEX OF THE PARAMETER IN THE DEVICE
from pythonosc import udp_client, dispatcher, osc_server
import threading

# Configuration
send_ip = "192.168.0.25"  # IP address of the OSC server
send_port = 11000         # Port to send messages to
listen_ip = "0.0.0.0"     # IP address to listen for replies (usually the same machine)
listen_port = 11001       # Port to listen for replies

# Send an OSC message
client = udp_client.SimpleUDPClient(send_ip, send_port)
track_id = 2
device_id = 1
message = "/live/device/get/parameters/name"
args = [track_id, device_id]
client.send_message(message, args)
print(f"Sent: {message} {args}")

# Handle incoming OSC replies
def handle_osc_reply(address, *args):
    print(f"Received reply from {address}: {args}")

# Start an OSC server to listen for replies
disp = dispatcher.Dispatcher()
disp.map("/live/device/get/parameters/name", handle_osc_reply)  # Map to the reply address

server = osc_server.ThreadingOSCUDPServer((listen_ip, listen_port), disp)
print(f"Listening for replies on {listen_ip}:{listen_port}")

# Start the OSC server in a separate thread
server_thread = threading.Thread(target=server.serve_forever)
server_thread.daemon = True
server_thread.start()

Sent: /live/device/get/parameters/name [2, 1]
Listening for replies on 0.0.0.0:11001


Received reply from /live/device/get/parameters/name: (2, 1, 'Device On', 'Filter Type', 'Filter Circuit - LP/HP', 'Filter Circuit - BP/NO/Morph', 'Slope', 'Frequency', 'Resonance', 'Morph', 'Drive', 'Env. Modulation', 'Env. Attack', 'Env. Release', 'LFO Amount', 'LFO Waveform', 'LFO Frequency', 'LFO Sync', 'LFO Sync Rate', 'LFO Stereo Mode', 'LFO Spin', 'LFO Phase', 'LFO Offset', 'LFO Quantize On', 'LFO Quantize Rate', 'S/C On', 'S/C Gain', 'S/C Mix')


In [ ]:
# MACHINE CODE FOR VOICE LEADING ALGO
import numpy as np
from scipy.optimize import linear_sum_assignment

def voice_leading(chord_a, chord_b):
    # Sort the chords
    chord_a_sorted = sorted(chord_a)
    chord_b_sorted = sorted(chord_b)
    
    # Create a cost matrix for pairings
    n = len(chord_a_sorted)
    m = len(chord_b_sorted)
    cost_matrix = np.zeros((n, m))
    
    for i, note_a in enumerate(chord_a_sorted):
        for j, note_b in enumerate(chord_b_sorted):
            # Cost is the minimum distance between two notes, considering wrap-around
            direct_move = abs(note_b - note_a)
            wrap_around = 12 - direct_move
            cost_matrix[i, j] = min(direct_move, wrap_around)
    
    # Solve the assignment problem
    row_indices, col_indices = linear_sum_assignment(cost_matrix)
    
    # Map notes from chord_a to chord_b
    new_chord = [chord_b_sorted[j] for i, j in zip(row_indices, col_indices)]
    
    return new_chord

# Example usage
chord_a = [60, 64, 67, 69]  # C major triad
chord_b = [62, 65, 69, 71]  # D minor triad

result = voice_leading(chord_a, chord_b)
print("Resulting chord with smooth voice leading:", result)

Received reply from /live/device/get/parameters/name: (2, 1, 'Device On', 'Filter Type', 'Filter Circuit - LP/HP', 'Filter Circuit - BP/NO/Morph', 'Slope', 'Frequency', 'Resonance', 'Morph', 'Drive', 'Env. Modulation', 'Env. Attack', 'Env. Release', 'LFO Amount', 'LFO Waveform', 'LFO Frequency', 'LFO Sync', 'LFO Sync Rate', 'LFO Stereo Mode', 'LFO Spin', 'LFO Phase', 'LFO Offset', 'LFO Quantize On', 'LFO Quantize Rate', 'S/C On', 'S/C Gain', 'S/C Mix')


Resulting chord with smooth voice leading: [71, 62, 65, 69]


In [ ]:
# CODE BY PABLO ADAPTED FROM THE PAPER OF AGRES
import numpy as np

def generate_mode_chords(valence):
    """
    Generate chord sequences based on valence and arousal inputs.

    Parameters:
    - valence (float): Emotional valence, a value between 0 and 1.
    - arousal (float): Emotional arousal, a value between 0 and 1.
    - delay (int): Duration (in seconds) to run the generation.

    Returns:
    - List of generated chord sequences for each bar.
    """
    # Define chord sets and modes
    chordlist = np.array([
        [60, 64, 55, 59],
        [62, 65, 57, 60],
        [64, 55, 59, 62],
        [60, 65, 57, 64],
        [55, 59, 62, 65],
        [57, 60, 64, 55],
        [59, 62, 65, 57]
    ])

    modes = ['lydian', 'ionian', 'mixolydian', 'dorian', 'aeolian', 'phrygian', 'locrian']
    mode_map = [
        [3, 6, 0, 3],    # Lydian
        [0, 3, 4, 0],    # Ionian
        [4, 0, 1, 4],    # Mixolydian
        [1, 4, 5, 1],    # Dorian
        [5, 1, 2, 5],    # Aeolian
        [2, 5, 6, 2],    # Phrygian
        [6, 2, 3, 6]     # Locrian
    ]

    # Build mode set (adjusting pitches by lowering 3 semitones)
    modeset = np.zeros((4, 4, 7))
    for i, mode_indices in enumerate(mode_map):
        for j in range(4):
            modeset[j, :, i] = chordlist[mode_indices[j], :]
    modeset -= 3

    # Determine mode based on valence
    mode_index = 7 - round(valence * 6) - 1
    mode_name = modes[int(mode_index)]
    print(f"Selected Mode: {mode_name}")

    # Generate chord sequence for the duration
    chord_sequence = []
    for bar in range(4):
        chords = modeset[bar, :, mode_index]
        chord_sequence.append(chords)
        print(f"Bar {bar + 1}: Chords {chords}")

    return chord_sequence


# Example usage
valence = 0.2 # Example valence
generated_chords = generate_mode_chords(valence)

Selected Mode: phrygian
Bar 1: Chords [61. 52. 56. 59.]
Bar 2: Chords [54. 57. 61. 52.]
Bar 3: Chords [56. 59. 62. 54.]
Bar 4: Chords [61. 52. 56. 59.]
